In [1]:
import random
import numpy as np
import time
import uuid
import csv
import os

In [2]:
POP_SIZE = 100
N_EVALS = 100000
N_REPLICATES = 30
MIN_LEN = 3
MAX_LEN = 100
CODON_MAX = 255

time_str = time.strftime('%Y%m%d_%H%M%S')
out_dir   = f'GE_results_{time_str}'
os.makedirs(out_dir, exist_ok=True)
csv_path  = os.path.join(out_dir, 'GE_results.csv')

In [3]:
def mkPenalizer(d_target, lam=10, deadzone=2):
    def _pen(genome):
        dev = abs(len(genome) - d_target)
        return 0 if dev <= deadzone else lam * dev
    return _pen

In [4]:
grammars = {
    'Koza': (
        {
            '<expr>': [
                ['(', '<expr>', '+', '<expr>', ')'],
                ['(', '<expr>', '-', '<expr>', ')'],
                ['(', '<expr>', '*', '<expr>', ')'],
                ['(', '<expr>', '/', '<expr>', ')'],
                ['x'],
                ['CONST']
            ],
            'CONST': [[str(v)] for v in [-1.0, -0.5, 0.5, 1.0]]
        },
        '<expr>',
        lambda tokens: (''.join(tokens), 'float')
    ),
    'Parity': (
        {
            '<expr>': [
                ['(', '<expr>', '^', '<expr>', ')'],
                ['(', '<expr>', '&', '<expr>', ')'],
                ['(', '<expr>', '|', '<expr>', ')'],
                ['~', '<expr>'],
                ['b0'], ['b1'], ['b2'], ['b3'], ['b4'], ['b5']
            ]
        },
        '<expr>',
        lambda tokens: (''.join(tokens), 'bool')
    ),
    'SantaFe': (
        {
            '<prog>': [['<prog>', '<cmd>'], ['<cmd>']],
            '<cmd>': [['0'], ['1'], ['2']],
        },
        '<prog>',
        lambda tokens: ([int(c) for c in ''.join(tokens)], 'actions')
    )
}

In [5]:
def loadSantafeMap(path='santafe_trail.txt'):
    rows, sx, sy = [], None, None
    with open(path) as f:
        for y, line in enumerate(f):
            row = []
            for x, c in enumerate(line.rstrip()):
                if c == '#': row.append(1)
                elif c in ('.','S'):
                    row.append(0)
                    if c == 'S':
                        sx, sy = x, y
            if row:
                rows.append(row)
    H, W = len(rows), len(rows[0])
    world = [cell for row in rows for cell in row]
    return world, W, H, sx, sy

In [6]:
def decodeGenome(genome, grammar, start_symbol, max_expansions=1000):
    output = [start_symbol]
    codon_ix = 0
    expansions = 0
    while expansions < max_expansions and any(sym in grammar for sym in output):
        for i, sym in enumerate(output):
            if sym in grammar:
                prods = grammar[sym]
                cod = genome[codon_ix % len(genome)]
                choice = prods[cod % len(prods)]
                output = output[:i] + choice + output[i+1:]
                codon_ix += 1
                expansions += 1
                break
    final = []
    for sym in output:
        if sym in grammar:
            for prod in grammar[sym]:
                if not any(s in grammar for s in prod):
                    final.extend(prod)
                    break
        else:
            final.append(sym)
    return final

def makeEvaluator(name, d_target, lam):
    grammar, start, post = grammars[name]
    pen = mkPenalizer(d_target, lam)

    if name == 'Koza':
        xs = [float(x) for x in np.linspace(-1,1,20)]
        def f(genome):
            try:
                tokens = decodeGenome(genome, grammar, start)
                expr, _ = post(tokens)
                func = lambda x: eval(expr, {'__builtins__':{}}, {'x': x})
                err = 0.0
                for x in xs:
                    try:
                        pred = func(x)
                        err += (pred - (x**4 + x**3 + x**2 + x))**2
                    except ZeroDivisionError:
                        return float('inf')
                return err + pen(genome)
            except Exception:
                return float('inf')
        return f

    elif name == 'Parity':
        def f(genome):
            try:
                tokens = decodeGenome(genome, grammar, start)
                expr, _ = post(tokens)
                fn = eval('lambda b0,b1,b2,b3,b4,b5: ' + expr,
                          {'__builtins__':{}}, {})
                errs = 0
                for i in range(64):
                    bits = [(i>>j)&1 for j in range(6)]
                    if fn(*bits) != (sum(bits) % 2):
                        errs += 1
                return errs + pen(genome)
            except Exception:
                return float('inf')
        return f

    else:
        world, W, H, SX, SY = loadSantafeMap()
        def f(genome):
            try:
                tokens = decodeGenome(genome, grammar, start)
                acts, _ = post(tokens)
                x, y, d = SX, SY, 0
                food = 0
                visited = world.copy()
                for i, a in enumerate(acts):
                    if i >= 400:
                        break
                    idx_ = (y % H)*W + (x % W)
                    if visited[idx_] == 1:
                        food += 1
                        visited[idx_] = 0
                    if a == 0:
                        dx, dy = [(0,1),(1,0),(0,-1),(-1,0)][d]
                        x += dx; y += dy
                    elif a == 1:
                        d = (d + 1) % 4
                    else:
                        d = (d - 1) % 4
                return -food + pen(genome)
            except Exception:
                return float('inf')
        return f

In [7]:
def genInd():
    return [random.randint(0, CODON_MAX)
            for _ in range(random.randint(MIN_LEN, MAX_LEN))]

def initPop():
    return [genInd() for _ in range(POP_SIZE)]

def tournamentSelection(pop, fit, k=3):
    return min(random.sample(pop, k), key=fit)

def cross(p1, p2, rate=0.9):
    if random.random() > rate:
        return p1[:], p2[:]
    c1 = random.randint(1, len(p1)-1)
    c2 = random.randint(1, len(p2)-1)
    o1 = p1[:c1] + p2[c2:]
    o2 = p2[:c2] + p1[c1:]
    for o in (o1, o2):
        while len(o) < MIN_LEN:
            o.append(random.randint(0, CODON_MAX))
    return o1[:MAX_LEN], o2[:MAX_LEN]

def mutate(genome, rate=0.1, p_len=0.3):
    g = genome[:]
    if random.random() < p_len:
        if random.random() < 0.5 and len(g) < MAX_LEN:
            g.append(random.randint(0, CODON_MAX))
        elif len(g) > MIN_LEN:
            del g[random.randrange(len(g))]
    for i in range(len(g)):
        if random.random() < rate:
            g[i] = random.randint(0, CODON_MAX)
    return g

In [8]:
def runGE(fit, name):
    gens = N_EVALS // POP_SIZE
    pop = initPop()
    for gen in range(1, gens+1):
        fits = [fit(ind) for ind in pop]
        best_idx = int(np.argmin(fits))
        best = fits[best_idx]
        if gen % 10 == 0:
            print(f'{name} Evals{gen*POP_SIZE:6d}: Best = {best:.6f}, Len = {len(pop[best_idx])}')
        new_pop = [pop[best_idx]]
        while len(new_pop) < POP_SIZE:
            p1 = tournamentSelection(pop, fit)
            p2 = tournamentSelection(pop, fit)
            c1, c2 = cross(p1, p2)
            new_pop.append(mutate(c1))
            if len(new_pop) < POP_SIZE:
                new_pop.append(mutate(c2))
        pop = new_pop
    fits = [fit(ind) for ind in pop]
    best_idx = int(np.argmin(fits))
    return pop[best_idx], fits[best_idx]

In [9]:
with open(csv_path, 'w', newline='') as f:
    writer = csv.DictWriter(f,
                            fieldnames=['run_id','benchmark','replicate','seed',
                                        'best','len_final','cpu_s'])
    writer.writeheader()
    """Dictionary s algoritmami"""
    benches = {
        #'Koza': makeEvaluator('Koza', 10, 10),
        'Parity': makeEvaluator('Parity', 64, 10),
        'SantaFe': makeEvaluator('SantaFe',100, 10),
    }
    for name, func in benches.items():
        for rep in range(N_REPLICATES):
            seed = random.randint(0, 2**31-1)
            random.seed(seed)
            np.random.seed(seed)
            start = time.perf_counter()
            best_genome, best_fit = runGE(func, name)
            cpu = time.perf_counter() - start
            writer.writerow({
                'run_id': uuid.uuid4(),
                'benchmark': name,
                'replicate': rep,
                'seed': seed,
                'best': round(best_fit, 6),
                'len_final': len(best_genome),
                'cpu_s': round(cpu, 4)
            })

print(f'\nResults written to {csv_path}')

Parity Evals  1000: Best = 32.000000, Len = 66
Parity Evals  2000: Best = 32.000000, Len = 66
Parity Evals  3000: Best = 32.000000, Len = 66
Parity Evals  4000: Best = 32.000000, Len = 66
Parity Evals  5000: Best = 32.000000, Len = 66
Parity Evals  6000: Best = 32.000000, Len = 66
Parity Evals  7000: Best = 32.000000, Len = 66
Parity Evals  8000: Best = 32.000000, Len = 66
Parity Evals  9000: Best = 31.000000, Len = 65
Parity Evals 10000: Best = 31.000000, Len = 65
Parity Evals 11000: Best = 28.000000, Len = 65
Parity Evals 12000: Best = 28.000000, Len = 65
Parity Evals 13000: Best = 28.000000, Len = 65
Parity Evals 14000: Best = 28.000000, Len = 65
Parity Evals 15000: Best = 28.000000, Len = 65
Parity Evals 16000: Best = 28.000000, Len = 65
Parity Evals 17000: Best = 28.000000, Len = 65
Parity Evals 18000: Best = 28.000000, Len = 65
Parity Evals 19000: Best = 28.000000, Len = 65
Parity Evals 20000: Best = 28.000000, Len = 65
Parity Evals 21000: Best = 28.000000, Len = 65
Parity Evals 